![Image of MLDO](https://ibm.box.com/shared/static/ammgpiy7zqfuhdm2s7hwxpjs573lupcv.png)

# 1. Obtain Data

In [54]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

In [55]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
spark = SparkSession(sc)

In [56]:
# The code was removed by Watson Studio for sharing.

[Row(life='15', Day-01='88', Day-02='104', Day-03='126', Day-04='94', Day-05='103', Day-06='107', Day-07='126', Day-08='95', Day-09='105', Day-10='120', Day-11='88', Day-12='114', Day-13='126', Day-14='90', Day-15='118', mid='14'),
 Row(life='9', Day-01='121', Day-02='96', Day-03='101', Day-04='98', Day-05='97', Day-06='93', Day-07='132', Day-08='118', Day-09='108', Day-10='139', Day-11='115', Day-12='125', Day-13='120', Day-14='129', Day-15='119', mid='9'),
 Row(life='8', Day-01='90', Day-02='95', Day-03='120', Day-04='126', Day-05='120', Day-06='113', Day-07='116', Day-08='133', Day-09='125', Day-10='125', Day-11='106', Day-12='129', Day-13='96', Day-14='99', Day-15='93', mid='4'),
 Row(life='6', Day-01='72', Day-02='60', Day-03='56', Day-04='65', Day-05='57', Day-06='64', Day-07='74', Day-08='84', Day-09='80', Day-10='84', Day-11='64', Day-12='65', Day-13='80', Day-14='70', Day-15='64', mid='6'),
 Row(life='12', Day-01='74', Day-02='67', Day-03='65', Day-04='74', Day-05='72', Day-06

# 2. Perform data exploration

In [57]:
data = data.dropna() # drop rows with missing valuesexprs = [col(column).alias(column.replace(' ', '_')) for column in data.columns]

In [58]:
data.describe()

DataFrame[summary: string, life: string, Day-01: string, Day-02: string, Day-03: string, Day-04: string, Day-05: string, Day-06: string, Day-07: string, Day-08: string, Day-09: string, Day-10: string, Day-11: string, Day-12: string, Day-13: string, Day-14: string, Day-15: string, mid: string]

In [59]:
from pyspark.sql.functions import col

In [60]:
data.show()

+----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+---+
|life|Day-01|Day-02|Day-03|Day-04|Day-05|Day-06|Day-07|Day-08|Day-09|Day-10|Day-11|Day-12|Day-13|Day-14|Day-15|mid|
+----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+---+
|  15|    88|   104|   126|    94|   103|   107|   126|    95|   105|   120|    88|   114|   126|    90|   118| 14|
|   9|   121|    96|   101|    98|    97|    93|   132|   118|   108|   139|   115|   125|   120|   129|   119|  9|
|   8|    90|    95|   120|   126|   120|   113|   116|   133|   125|   125|   106|   129|    96|    99|    93|  4|
|   6|    72|    60|    56|    65|    57|    64|    74|    84|    80|    84|    64|    65|    80|    70|    64|  6|
|  12|    74|    67|    65|    74|    72|    73|    80|    80|    80|    80|    80|    61|    80|    80|    80|  6|
|  14|    90|   104|   107|    99|   126|   119|   102|    86|    88|   

In [61]:

df2 = data.select("life","Day-01","Day-02","Day-03","Day-04","Day-05","Day-06","Day-07","Day-08","Day-09","Day-10","Day-11","Day-12","Day-13","Day-14","Day-15","mid")
df2.take(5)

[Row(life='15', Day-01='88', Day-02='104', Day-03='126', Day-04='94', Day-05='103', Day-06='107', Day-07='126', Day-08='95', Day-09='105', Day-10='120', Day-11='88', Day-12='114', Day-13='126', Day-14='90', Day-15='118', mid='14'),
 Row(life='9', Day-01='121', Day-02='96', Day-03='101', Day-04='98', Day-05='97', Day-06='93', Day-07='132', Day-08='118', Day-09='108', Day-10='139', Day-11='115', Day-12='125', Day-13='120', Day-14='129', Day-15='119', mid='9'),
 Row(life='8', Day-01='90', Day-02='95', Day-03='120', Day-04='126', Day-05='120', Day-06='113', Day-07='116', Day-08='133', Day-09='125', Day-10='125', Day-11='106', Day-12='129', Day-13='96', Day-14='99', Day-15='93', mid='4'),
 Row(life='6', Day-01='72', Day-02='60', Day-03='56', Day-04='65', Day-05='57', Day-06='64', Day-07='74', Day-08='84', Day-09='80', Day-10='84', Day-11='64', Day-12='65', Day-13='80', Day-14='70', Day-15='64', mid='6'),
 Row(life='12', Day-01='74', Day-02='67', Day-03='65', Day-04='74', Day-05='72', Day-06

In [62]:


from pyspark.sql.types import FloatType
df2 = df2.withColumn("life", df2["life"].cast(FloatType()))
df2 = df2.withColumn("Day-01", df2["Day-01"].cast(FloatType()))
df2 = df2.withColumn("Day-02", df2["Day-02"].cast(FloatType()))
df2 = df2.withColumn("Day-03", df2["Day-03"].cast(FloatType()))
df2 = df2.withColumn("Day-04", df2["Day-04"].cast(FloatType()))
df2 = df2.withColumn("Day-05", df2["Day-05"].cast(FloatType()))
df2 = df2.withColumn("Day-06", df2["Day-06"].cast(FloatType()))
df2 = df2.withColumn("Day-07", df2["Day-07"].cast(FloatType()))
df2 = df2.withColumn("Day-08", df2["Day-08"].cast(FloatType()))
df2 = df2.withColumn("Day-09", df2["Day-09"].cast(FloatType()))
df2 = df2.withColumn("Day-10", df2["Day-10"].cast(FloatType()))
df2 = df2.withColumn("Day-11", df2["Day-11"].cast(FloatType()))
df2 = df2.withColumn("Day-12", df2["Day-12"].cast(FloatType()))
df2 = df2.withColumn("Day-13", df2["Day-13"].cast(FloatType()))
df2 = df2.withColumn("Day-14", df2["Day-14"].cast(FloatType()))
df2 = df2.withColumn("Day-15", df2["Day-15"].cast(FloatType()))
df2 = df2.withColumn("mid", df2["mid"].cast(FloatType()))

In [63]:
#df2 = df2.select("fDay1","fDay2","fDay3","fDay4","fDay5","fDay6","fDay7","fDay8","fDay9","fDay10","fDay11","fDay12","fDay13","fDay14","fDay15","fmid")
df2.take(5)
features=["life","Day-01","Day-02","Day-03","Day-04","Day-05","Day-06","Day-07","Day-08","Day-09","Day-10","Day-11","Day-12","Day-13","Day-14","Day-15"]

In [64]:
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
#stages=[]
#numericCols = df2
#label_stringIdx = StringIndexer(inputCol = 'mid', outputCol = 'label')
#assembler = VectorAssembler(inputCols=numericCols, outputCol="features")
#stages += [assembler]

In [65]:
from pyspark.ml.feature import VectorAssembler

In [66]:

lr_data = df2.select(col("mid").alias("label"), *features )
lr_data.printSchema()

root
 |-- label: float (nullable = true)
 |-- life: float (nullable = true)
 |-- Day-01: float (nullable = true)
 |-- Day-02: float (nullable = true)
 |-- Day-03: float (nullable = true)
 |-- Day-04: float (nullable = true)
 |-- Day-05: float (nullable = true)
 |-- Day-06: float (nullable = true)
 |-- Day-07: float (nullable = true)
 |-- Day-08: float (nullable = true)
 |-- Day-09: float (nullable = true)
 |-- Day-10: float (nullable = true)
 |-- Day-11: float (nullable = true)
 |-- Day-12: float (nullable = true)
 |-- Day-13: float (nullable = true)
 |-- Day-14: float (nullable = true)
 |-- Day-15: float (nullable = true)



# 3. Train data

In [67]:
#split data to train/test

In [68]:
(training, test) = lr_data.randomSplit([.7, .3])

In [69]:
#creating ML pipeline 

In [70]:
from pyspark.ml.regression import LinearRegression

vectorAssembler = VectorAssembler(inputCols=features, outputCol="features")

lr = LinearRegression(maxIter=10, regParam=.01)

stages = [vectorAssembler, lr]
pipeline = Pipeline(stages=stages)

In [71]:
model = pipeline.fit(training)
prediction = model.transform(test)

In [72]:
prediction.show()

+-----+----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+--------------------+------------------+
|label|life|Day-01|Day-02|Day-03|Day-04|Day-05|Day-06|Day-07|Day-08|Day-09|Day-10|Day-11|Day-12|Day-13|Day-14|Day-15|            features|        prediction|
+-----+----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+--------------------+------------------+
|  3.0| 5.0|  60.0|  80.0|  70.0|  61.0|  58.0|  81.0|  71.0|  79.0|  79.0|  67.0|  64.0|  71.0|  75.0|  63.0|  73.0|[5.0,60.0,80.0,70...|4.0328870314935585|
|  3.0| 5.0|  75.0|  63.0|  73.0|  65.0|  66.0|  67.0|  61.0|  69.0|  66.0|  72.0|  60.0|  70.0|  53.0|  74.0|  66.0|[5.0,75.0,63.0,73...|4.1172172642723135|
|  3.0| 5.0|  79.0|  68.0|  70.0|  74.0|  71.0|  76.0|  76.0|  69.0|  68.0|  70.0|  71.0|  58.0|  77.0|  55.0|  69.0|[5.0,79.0,68.0,70...|3.6777531889444046|
|  3.0| 5.0|  88.0|  70.0|  76.0|  85.0|  88.0|  73.

In [73]:
model.transform(test)\
    .select("label", "prediction")\
    .show()

+-----+------------------+
|label|        prediction|
+-----+------------------+
|  3.0|4.0328870314935585|
|  3.0|4.1172172642723135|
|  3.0|3.6777531889444046|
|  3.0|3.7127736934361475|
|  3.0| 4.540546349404495|
|  3.0|3.9951937147311143|
|  3.0| 3.907528715949877|
|  3.0|4.2197655202815945|
|  3.0| 4.382724602872386|
|  3.0|3.5647868291521325|
|  3.0| 4.698680898432499|
|  3.0|3.7489707111076918|
|  3.0| 4.649569200982652|
|  3.0| 4.730501086745998|
|  3.0|  4.26677431750848|
|  3.0| 4.640885720238192|
|  3.0| 4.780620226996942|
|  3.0| 4.207140708835986|
|  3.0|4.4124692612866205|
|  3.0| 4.736065701032651|
+-----+------------------+
only showing top 20 rows



In [74]:
from pyspark.ml.evaluation import RegressionEvaluator
eval = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")

# Root Mean Square Error
rmse = eval.evaluate(prediction)
print("RMSE: %.3f" % rmse)

# Mean Square Error
mse = eval.evaluate(prediction, {eval.metricName: "mse"})
print("MSE: %.3f" % mse)

# Mean Absolute Error
mae = eval.evaluate(prediction, {eval.metricName: "mae"})
print("MAE: %.3f" % mae)

# r2 - coefficient of determination
r2 = eval.evaluate(prediction, {eval.metricName: "r2"})
print("r2: %.3f" %r2)


RMSE: 1.758
MSE: 3.091
MAE: 1.469
r2: 0.659


# 4. Deploy Predictive Model

In [75]:
!pip install watson-machine-learning-client --upgrade

  Using cached https://files.pythonhosted.org/packages/b6/47/32daed1eb2169e93108480d7524fffa533f2375d8729d597c4b50cc2a6e8/watson_machine_learning_client-1.0.376-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/18/b0/8146a4f8dd402f60744fa380bc73ca47303cccf8b9190fd16a827281eac2/certifi-2019.9.11-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/0f/b1/02eebed49c754b01b17de7705caa8c4ceecfb4f926cdafc220c863584360/lomond-0.3.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/05/f2/764a5d530cf143ded9bc95216edb6e258c6554511e78de7c250557e8f3ed/tqdm-4.37.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/52/3f/f6a428599e0d4497e1595030965b5ba455fd8ade6e977e3c819973c4b41d/pandas-0.25.3-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/e0/da/55f51ea951e1b7c63a579c09dd7db825bb730ec1fe9c0180fc77bfb31448/urllib3-1.25.6-py2.py3-none-any.whl
  Using cached https://

In [76]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [77]:
# The code was removed by Watson Studio for sharing.

In [78]:
wml_client = WatsonMachineLearningAPIClient(wml_credentials)

In [79]:
model_props = {wml_client.repository.ModelMetaNames.AUTHOR_NAME: "Nerav Doshi",  
               wml_client.repository.ModelMetaNames.NAME: "Predictive Maintenance Model"}

published_model = wml_client.repository.store_model(model=model, pipeline=pipeline, meta_props=model_props, training_data=training)

In [80]:
import json
published_model_uid = wml_client.repository.get_model_uid(published_model)
model_details = wml_client.repository.get_details(published_model_uid)
print(json.dumps(model_details, indent=2))

{
  "metadata": {
    "guid": "e1bf9467-dec0-4cf5-a4cf-47d99fb45a35",
    "url": "https://us-south.ml.cloud.ibm.com/v3/wml_instances/ce211bae-2d5c-4278-8eee-3b061501d712/published_models/e1bf9467-dec0-4cf5-a4cf-47d99fb45a35",
    "created_at": "2019-11-07T20:05:36.334Z",
    "modified_at": "2019-11-07T20:05:36.441Z"
  },
  "entity": {
    "runtime_environment": "spark-2.3",
    "learning_configuration_url": "https://us-south.ml.cloud.ibm.com/v3/wml_instances/ce211bae-2d5c-4278-8eee-3b061501d712/published_models/e1bf9467-dec0-4cf5-a4cf-47d99fb45a35/learning_configuration",
    "author": {
      "name": "Nerav Doshi"
    },
    "name": "Predictive Maintenance Model",
    "label_col": "label",
    "learning_iterations_url": "https://us-south.ml.cloud.ibm.com/v3/wml_instances/ce211bae-2d5c-4278-8eee-3b061501d712/published_models/e1bf9467-dec0-4cf5-a4cf-47d99fb45a35/learning_iterations",
    "training_data_schema": {
      "fields": [
        {
          "metadata": {
            "modeling_

In [81]:
#List all models
wml_client.repository.list_models()

------------------------------------  ----------------------------  ------------------------  -----------------
GUID                                  NAME                          CREATED                   FRAMEWORK
e1bf9467-dec0-4cf5-a4cf-47d99fb45a35  Predictive Maintenance Model  2019-11-07T20:05:36.334Z  mllib-2.3
46225946-5d85-4337-beba-e184859a47c0  Predictive Maintenance Model  2019-11-07T19:03:33.937Z  mllib-2.3
641e606b-33d2-4a02-a709-438d37f583c3  Predictive Maintenance Model  2019-11-07T02:42:42.908Z  mllib-2.3
7a1a5b3c-5129-448c-aebb-04cbabf05b9f  kidney                        2019-02-14T22:40:35.983Z  spss-modeler-18.1
ed2a90e7-95e6-43fe-a7ca-b97f40be6ece  demo 2                        2019-02-14T19:19:44.039Z  spss-modeler-18.1
78b13ca9-747c-4eff-af88-eb157fbe3948  ewpo                          2019-02-13T18:20:56.565Z  spss-modeler-18.1
b225e512-1335-4d88-a3b6-d381e0a3c77c  drug study 1                  2019-02-13T18:00:29.009Z  spss-modeler-18.1
9b6f03fc-e8fe-4954-b2a1-

In [82]:
created_deployment = wml_client.deployments.create(published_model_uid, name="Predictive Maintenance Model Py36 Online Deployment")



#######################################################################################

Synchronous deployment creation for uid: 'e1bf9467-dec0-4cf5-a4cf-47d99fb45a35' started

#######################################################################################


INITIALIZING
DEPLOY_IN_PROGRESS....
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='9e4a67d8-1468-49fc-b068-569c01e6b74c'
------------------------------------------------------------------------------------------------




# 5. Score and test

In [83]:
scoring_endpoint = wml_client.deployments.get_scoring_url(created_deployment)

print(scoring_endpoint)

https://us-south.ml.cloud.ibm.com/v3/wml_instances/ce211bae-2d5c-4278-8eee-3b061501d712/deployments/9e4a67d8-1468-49fc-b068-569c01e6b74c/online


In [89]:
wml_client.deployments.list()

------------------------------------  ---------------------------------------------------  ------  --------------  ------------------------  -----------------  -------------
GUID                                  NAME                                                 TYPE    STATE           CREATED                   FRAMEWORK          ARTIFACT TYPE
9e4a67d8-1468-49fc-b068-569c01e6b74c  Predictive Maintenance Model Py36 Online Deployment  online  DEPLOY_SUCCESS  2019-11-07T20:05:41.170Z  mllib-2.3          model
6b488d5b-e2c6-4ffa-9cda-e64c931bb195  api2                                                 online  DEPLOY_SUCCESS  2019-02-15T00:25:31.594Z  spss-modeler-18.1  model
fe6ec487-0ca0-4058-9de7-39ce21f173c2  im an api                                            online  DEPLOY_SUCCESS  2019-02-14T22:41:30.317Z  spss-modeler-18.1  model
be71378d-1e7a-48c6-835f-f0885579d249  deployed api                                         online  DEPLOY_SUCCESS  2019-02-14T19:21:35.598Z  spss-modeler-

In [88]:
wml_client.deployments.delete('f7e4b59c-f394-4ff4-a7e4-ae70f16db749')

'SUCCESS'

## Test model with a REST API call (we will need this for future steps in the notebooks)

This step demonstrates an "internal REST API" call to test the model (for an unpublished model). Notice that we are using internal values for the model endpoint and token. See documentation for external REST call syntax. An external REST call will have a different end point and will require authentication. 

In [ ]:
scoring_payload = {
    "fields": [
    "life",
    "Day-01",
    "Day-02",
    "Day-03",
    "Day-04",
    "Day-05",
    "Day-06",
    "Day-07",
    "Day-08",
    "Day-09",
    "Day-10",
    "Day-11",
    "Day-12",
    "Day-13",
    "Day-14",
    "Day-15",
    ],
    "values": [ [5,60,80,70,61,56,78,89,67,89,45,35,80,78,98,73] ]
} 

In [ ]:
predictions = wml_client.deployments.score(scoring_endpoint, scoring_payload)

In [ ]:
print ('Prediction = {}'.format(predictions))